In [1]:
import matplotlib.pyplot as plt
import geopandas
import shapely.wkt
from shapely import geometry
import pandas as pd
import numpy as np
import csv
import re

In [2]:
# Creates WKT Polygons
def getPolygons(id_list):
    all_polys = {}

    for poly_id in id_list:
        tmp = df2[df2['POLYGON_ID'] == poly_id]

        long = tmp['LONGITUDE'].astype(str).values
        lat = tmp['LATITUDE'].astype(str).values

        pointList = list(zip(tmp['LONGITUDE'], tmp['LATITUDE']))

        if len(pointList) == 1:
            poly = geometry.Point([pointList[0][0], pointList[0][1]])
        elif len(pointList) == 2:
            poly = geometry.LineString([[p[0], p[1]] for p in pointList])
        else:
            poly = geometry.Polygon([[p[0], p[1]] for p in pointList])

        all_polys[poly_id] = poly
        
    return all_polys

# Gets Poylgon ID from NOTAM ID
def getIds(rec_ids, poly_df):
    id_list = []
    for row in rec_ids['NOTAM_REC_ID']:
        for item in poly_df[poly_df['NOTAM_REC_ID'] == row]['POLYGON_ID'].values:
            id_list.append(item)
            
    return id_list

In [3]:
#This is a utility function for converting distance values using different units
def conv_dist(distance_value, units_value):
    # Determine the conversion factor for the specified units (meters are required for this projection)
    if units_value == "mi":
        unit_factor = 1609.344
    elif units_value == "km":
        unit_factor = 1000.0
    elif units_value == "ft":
        unit_factor = 0.3048
    elif units_value == "nm":
        unit_factor = 1852
    elif units_value == "m":
        unit_factor = 1
    else:  # Bad units
        unit_factor = 0

    return distance_value * unit_factor

#This function creates geospatial circle(s) based on center, radius and unit values in the dataset
def gen_geocircle(input_df, key_col, center_col, radius_col, units_col):

    # Convert point data to geopandas dataframe
#    pointsdf = input_df[[key_col, center_col, radius_col, units_col]]

    working_cols = [key_col] + [center_col] + [radius_col] + [units_col]
    return_cols = [key_col] + ["buffer"]
    
    pointsdf = input_df[working_cols]
    
#    pointsdf[center_col] = gpd.GeoSeries.from_wkt(pointsdf[center_col])
    gdf_pts = geopandas.GeoDataFrame(pointsdf, geometry=center_col)

    # Add CRS (start with WGS84 to match lat/lon values)
    gdf_pts.set_crs(epsg=4326, inplace=True)

    #Prepare projection (North America Lambert Conformal Conic)
    # This projection is equidistant for measuring between points.
    # Units are in meters
    projout = '+proj=lcc +lat_1=20 +lat_2=60 +lat_0=40 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m no_defs'

    # Convert to Lambert projection
    gdf_pts = gdf_pts.to_crs(projout)

    # -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
    gdf_pts[units_col] = gdf_pts[units_col].str.lower()
    gdf_pts["dist"] = 0

    # -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
    for pt_index,pt_row in gdf_pts.iterrows():
        dvalue = conv_dist(pt_row[2], pt_row[3])

        gdf_pts.loc[pt_index,'dist'] = dvalue

#    gdf2 = pd.merge(gdf_pts[["locid","ArptLocation"]], params[["locid","dist"]], on='locid')

    gdf_pts["buffer"] = gdf_pts[center_col].buffer(gdf_pts['dist'])

    # -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
    gdf_circle = gdf_pts[return_cols]

    gdf_circle = geopandas.GeoDataFrame(gdf_circle, geometry='buffer')

    gdf_circle = gdf_circle.to_crs(epsg=4326)

    
    return gdf_circle


In [4]:
#This function determines takes two sets of geospatial objects and determines which ones interact
def find_interactions(geom_set1_df, geom_set2_df,
                      set1_key_cols, set2_key_cols,
                      set1_geometry_col, set2_geometry_col
                      ):

    set1_working_cols = set1_key_cols + [set1_geometry_col]
    set2_working_cols = set2_key_cols + [set2_geometry_col]
    
    # Convert point data to geopandas dataframe

    gdf1 = geom_set1_df[set1_working_cols]
#    gdf1[set1_geometry_col] = gpd.GeoSeries.from_wkt(gdf1[set1_geometry_col])
    gdf1 = geopandas.GeoDataFrame(gdf1, geometry=set1_geometry_col)

    # Add CRS (start with WGS84 to match lat/lon values)
    gdf1.set_crs(epsg=4326, inplace=True)
    gdf1_type = gdf1.loc[0, set1_geometry_col].geom_type

    #Convert polygon data to geopandas dataframe
    gdf2 = geom_set2_df[set2_working_cols]
#    gdf2[set2_geometry_col] = gpd.GeoSeries.from_wkt(gdf2[set2_geometry_col])
    gdf2 = geopandas.GeoDataFrame(gdf2, geometry=set2_geometry_col)

    # Add CRS (start with WGS84 to match lat/lon values)
    gdf2.set_crs(epsg=4326, inplace=True)

    gdf2_type = gdf2.loc[0, set2_geometry_col].geom_type

    #If datasets are mixed (one polygon and one linestring), ensure polygons are gdf1
    if (gdf1_type == 'LineString' or gdf1_type == 'Point') and gdf2_type == 'Polygon':
        gdf_temp = gdf1
        gdf1 = gdf2
        gdf2 = gdf_temp
        gdf2_type = gdf1_type
        gdf1_type = 'Polygon'
        keys_temp = set1_key_cols
        set1_key_cols = set2_key_cols
        set2_key_cols = keys_temp
        geom_temp = set1_geometry_col
        set1_geometry_col = set2_geometry_col
        set2_geometry_col = geom_temp
        
    # Convert to new equidistant projection

    #Prepare projection (North America Lambert Conformal Conic)
    # This projection is equidistant for measuring between points.
    # Units are in meters
    projout = '+proj=lcc +lat_1=20 +lat_2=60 +lat_0=40 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m no_defs'

    # Convert to Lambert projection
    gdf1 = gdf1.to_crs(projout)

    # Convert to Lambert projection
    gdf2 = gdf2.to_crs(projout)

#    if gdf2_type == 'Point':
#        results = pd.DataFrame(columns=(set1_key_cols + ['Within_polygon']))
#    else:
    results = pd.DataFrame(columns=(set1_key_cols + set2_key_cols + ['Interaction']))
    
    df_index = 0
    
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
    for gdf1_index,gdf1_row in gdf1.iterrows():
        for gdf2_index,gdf2_row in gdf2.iterrows():
            if gdf1_type == 'Polygon' and gdf2_type == 'LineString':
                interaction = gdf1.loc[gdf1_index, set1_geometry_col].intersects(gdf2.loc[gdf2_index, set2_geometry_col])
                if interaction:
                    start_pt, end_pt = gdf2.loc[gdf2_index, set2_geometry_col].boundary
                    if gdf1.loc[gdf1_index, set1_geometry_col].contains(gdf2.loc[gdf2_index, set2_geometry_col]):
                        interact_str = ['Contains']
                    elif gdf1.loc[gdf1_index, set1_geometry_col].contains(start_pt):
                        interact_str = ['Start']
                    elif gdf1.loc[gdf1_index, set1_geometry_col].contains(end_pt):
                        interact_str = ['End']
                    else:
                        interact_str = ['Intersect']

                    results.loc[df_index] = np.concatenate((gdf1.loc[gdf1_index, set1_key_cols].values,
                                            gdf2.loc[gdf2_index, set2_key_cols].values,
                                            interact_str),axis=None)
                    df_index += 1
            elif gdf1_type == 'LineString' and gdf2_type == 'LineString':
                interaction = gdf1.loc[gdf1_index, set1_geometry_col].intersects(gdf2.loc[gdf2_index, set2_geometry_col])
                if interaction:
                    interact_str = ['Intersect']

                    results.loc[df_index] = np.concatenate((gdf1.loc[gdf1_index, set1_key_cols].values,
                                            gdf2.loc[gdf2_index, set2_key_cols].values,
                                            interact_str),axis=None)
                    df_index += 1
            elif gdf1_type == 'Polygon' and gdf2_type == 'Polygon':
                interaction = gdf1.loc[gdf1_index, set1_geometry_col].intersects(gdf2.loc[gdf2_index, set2_geometry_col])
                if interaction:
                    if gdf1.loc[gdf1_index, set1_geometry_col].contains(gdf2.loc[gdf2_index, set2_geometry_col]):
                        interact_str = ['Contains']
                    else:
                        interact_str = ['Intersect']

                    results.loc[df_index] = np.concatenate((gdf1.loc[gdf1_index, set1_key_cols].values,
                                            gdf2.loc[gdf2_index, set2_key_cols].values,
                                            interact_str),axis=None)
                    df_index += 1
            elif gdf1_type == 'Polygon' and gdf2_type == 'Point':            
                if gdf2.loc[gdf2_index, set2_geometry_col].within(gdf1.loc[gdf1_index, set1_geometry_col]):
                    results.loc[df_index] = np.concatenate((gdf1.loc[gdf1_index, set1_key_cols].values,
                                                            gdf2.loc[gdf2_index, set2_key_cols].values,
                                                            "Contained"),axis=None)
                    df_index += 1

            else:
                print('no match')
                
                
    return results


In [5]:
# Read FIR Codes
df = pd.read_csv('data/NA_FIR_Codes.csv')

# Read Vertices
df2 = pd.read_csv('data/vertices_20220621.csv')

# Read Spaceports
df3 = pd.read_csv ('data/spaceports_20201027.csv')

# Read pickle file with topics and augmented text
#df4 = pd.read_pickle("data/allData.pkl")

# Read Polygon File
df5 = pd.read_csv('data/polygon_20201027.csv')

# Read Basemap Shapefile
states = geopandas.read_file('data/bound_p.shx')

In [6]:
#Convert the latitude/longtitude values to geospatial points
spaceport_df = df3

spaceport_df['FacilityLocation'] = [geometry.Point(xy) for xy in zip(spaceport_df['LONGITUDE'], spaceport_df['LATITUDE'])]


spaceport_df['radius'] = 100
spaceport_df['units'] = 'nm'

spaceport_df = spaceport_df[spaceport_df['SPACEPORT_REC_ID'] > 1]

sp_df_2 = gen_geocircle(spaceport_df, 'SPACEPORT_REC_ID', 'FacilityLocation', 'radius', 'units')

C:\Users\saebasan\anaconda3\envs\spatial\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [7]:
# Get IDs for Candidate NOTAMs
#id_list = getIds(filterE, df5)

#id_list = [39421, 52823, 60100, 77638]
id_list = [13425, 13426, 13427, 13428, 13434, 16347, 17108, 17109, 17110, 17115, 17117, 17118, 17119]

#id_list = [21892, 21894, 21896, 21898, 21906, 21908, 21912, 21913, 21916, 21917, 21927, 21928, 21929, 21931, 21933, 21935, 21937, 21942, 21943, 21945, 21946, 21957, 21958, 21960, 21962, 21966, 21967, 22081, 22082, 22089, 23055, 23060, 23067, 23068, 23071]

# Get Polygons for Candidate NOTAMs
P = getPolygons(id_list)

In [8]:
test2 = pd.DataFrame(P, index=['boundary']).rename_axis('polygon_id', axis=1).transpose().reset_index()

sp_df_2.reset_index(inplace=True)

interactions_df = find_interactions(sp_df_2, test2,
                      ['SPACEPORT_REC_ID'], ['polygon_id'],
                      'buffer', 'boundary'
                      )

In [9]:
interactions_df

,SPACEPORT_REC_ID,polygon_id,Interaction
0,9,13427,Contained


In [10]:
assert False

AssertionError: 

In [ ]:
sp_df_2

In [ ]:
states.plot(cmap='Pastel2', figsize=(30, 10))

plt.plot(df3['longitude_degrees'], df3['latitude_degrees'], marker="o", markeredgecolor="blue", markerfacecolor="black", markersize=.07, linestyle="None")

for item in P:
    x,y = item.exterior.xy
    plt.plot(x,y, linestyle="dashdot")

plt.plot(df2['LONGITUDE'], df2['LATITUDE'], marker="o", markeredgecolor="red", markerfacecolor="yellow", linestyle="None")

In [ ]:
states = geopandas.read_file('data/bound_p.shx')

#df = pd.read_csv ('data/faa_artcc_polygons.csv')
df2 = pd.read_csv ('data/spaceports.csv')
#df3 = pd.read_csv ('data/AirportData_Clean_20210629_Geocoded.csv')

#P = [shapely.wkt.loads(h) for h in df['boundary']]

In [ ]:
P = [h for h in sp_df_2['buffer']]

In [ ]:
states.plot(cmap='Pastel2', figsize=(30, 10))

plt.plot(df3['longitude_degrees'], df3['latitude_degrees'], marker="o", markeredgecolor="blue", markerfacecolor="black", markersize=.07, linestyle="None")

for item in P:
    x,y = item.exterior.xy
    plt.plot(x,y, linestyle="dashdot")

plt.plot(df2['LONGITUDE'], df2['LATITUDE'], marker="o", markeredgecolor="red", markerfacecolor="yellow", linestyle="None")

# Dynamic plotting

In [ ]:
# Plot Base Map
states.plot(cmap='Pastel2', figsize=(30, 10))

# Plot NOTAM Polygons
for item in P.values():
    #if P:
     #   continue
    if item.geom_type != 'Polygon':
        plt.plot(*item.xy)
    else:
        x,y = item.exterior.xy
        plt.plot(x,y)

# Plot Spaceport Locations
#plt.plot(df3['LONGITUDE'], df3['LATITUDE'], marker="o", markeredgecolor="red", markerfacecolor="yellow", linestyle="None")

# Set Zoom
#plt.axis([-83, -78, 38, 45])
plt.axis([-162, -60, 10, 50])

In [ ]:
type(P)

In [ ]:
P.keys()

In [ ]:
test1 = pd.DataFrame.from_dict(P, orient='index', columns = ['boundary'])

In [ ]:
test1.columns = ['Polygon_ID','boundary']

In [ ]:
test1


In [ ]:
test2